# Deep Deterministic Policy Gradients

## Imports

In [2]:
import copy

import gym
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import logger, wrappers
import pandas as pd

from actor_critic import Actor, Critic
from utils import OUNoise, ReplayBuffer
sns.set()
sns.set_style("whitegrid")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class DDPG(object):
    def __init__(self, env, state_dim, action_dim, min_action, max_action, discount=0.99, tau=0.001,
                 start_timesteps=1000, expl_noise=.1):
        """DDPG Agent

        :param env:
        :param state_dim:
        :param action_dim:
        :param min_action:
        :param max_action:
        :param discount:
        :param tau:
        :param start_timesteps:
        :param expl_noise:
        :returns:
        :rtype:

        """
        self.actor = Actor(
            state_dim,
            action_dim,
            min_action,
            max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(
            self.actor.parameters(), lr=0.0001)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(
            self.critic.parameters(), lr=0.001)

        self.actor.float()
        self.actor_target.float()
        self.critic.float()
        self.critic_target.float()

        self.discount = discount
        self.tau = tau

        self.replay_buffer = ReplayBuffer(state_dim, action_dim)
        self.last_state = None
        self.last_action = None
        self.start_timesteps = start_timesteps
        self.expl_noise = expl_noise
        self.env = env
        self.min_action = min_action
        self.max_action = max_action
        self.action_dim = action_dim

        self.exploration_mu = 0
        self.exploration_theta = 0.15
        self.exploration_sigma = 0.2
        self.noise = OUNoise(
            self.action_dim,
            self.exploration_mu,
            self.exploration_theta,
            self.exploration_sigma)
        self.epsilon = 1.0
        self.epsilon_decay = 1e-6

    def act(self, state, warmup=False):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)

        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state)
        if warmup:
            action = self.env.action_space.sample()
        else:
            action = self.actor(state).cpu().data.numpy().flatten()
        action += self.noise.sample() * self.epsilon
        action = action.clip(self.min_action, self.max_action)

        self.last_action = action
        self.last_state = state.cpu()

        return self.last_action

    def update(self, next_state, reward, done, batch_size):
        if self.last_action is not None:
            self.replay_buffer.add(
                self.last_state,
                self.last_action,
                next_state,
                reward,
                done)

        if self.replay_buffer.size > batch_size == 0:
            self.train(batch_size)

        self.last_state = next_state

    def train(self, batch_size=100):
        state, action, next_state, reward, not_done = self.replay_buffer.sample(
            batch_size)

        # Compute the target Q value
        target_Q = self.critic_target(
            next_state, self.actor_target(next_state))
        target_Q = reward + (not_done * self.discount * target_Q).detach()

        # Get current Q estimate
        current_Q = self.critic(state, action)

        # Compute critic loss
        critic_loss = F.mse_loss(current_Q, target_Q)

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Compute actor loss
        actor_loss = -self.critic(state, self.actor(state)).mean()

        # Optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update the frozen target models
        for param, target_param in zip(
                self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(
                self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(
                self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(
                self.tau * param.data + (1 - self.tau) * target_param.data)

        self.epsilon -= self.epsilon_decay

    def reset_noise(self):
        self.noise.reset()

    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic")
        torch.save(
            self.critic_optimizer.state_dict(),
            filename + "_critic_opt")
        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(
            self.actor_optimizer.state_dict(),
            filename + "_actor_opt")

    def load(self, filename):
        self.critic.load_state_dict(torch.load(filename + "_critic"))
        self.critic_optimizer.load_state_dict(
            torch.load(filename + "_critic_opt"))
        self.actor.load_state_dict(torch.load(filename + "_actor"))
        self.actor_optimizer.load_state_dict(
            torch.load(filename + "_actor_opt"))

## MountainCarContinuous-v0

In [5]:
if __name__ == "__main__":
    name_file = "ddpg.csv"
    env = gym.make("MountainCarContinuous-v0")
    agent = DDPG(
        env=env,
        state_dim=env.observation_space.shape[0],
        action_dim=env.action_space.shape[0],
        max_action=float(
            env.action_space.high[0]),
        min_action=float(
            env.action_space.low[0]))  # ,tau=0.005)

    outdir = 'mountain-continuous/random-agent-results'
    envm = wrappers.Monitor(
        env,
        directory=outdir,
        force=True,
        video_callable=False)
    env.seed(0)

    episode_count = 1000

    reward = 0
    done = False
    env.verbose = True
    np.random.seed(5)
    rsum = 0
    all_rsums = []
    for i in range(episode_count):
        obs = envm.reset()
        agent.reset_noise()
        env.verbose = (i % 100 == 0 and i > 0)  # afficher 1 episode sur 100
        if env.verbose:
            env.render()
        j = 0
        rsum = 0
        episode_reward = 0
        episode_timesteps = 0
        while True:
            episode_timesteps += 1
            warmup = i < 50
            action = agent.act(obs, warmup=warmup)

            obs, reward, done, _ = envm.step(action)
            rsum += reward
            j += 1
            done_bool = float(
                done) if episode_timesteps < env._max_episode_steps else 1

            agent.update(obs, reward, done_bool, 64)
            if env.verbose:
                env.render()
            if done:
                print(
                    "Episode : " +
                    str(i) +
                    " rsum=" +
                    str(rsum) +
                    ", " +
                    str(j) +
                    " actions")
                all_rsums.append(rsum)
                break

    print("done")
    print("Average rsum : {}, std : {}".format(np.mean(all_rsums), np.std(all_rsums)))
    env.close()

    reward_a2c = all_rsums[:]
    results = pd.DataFrame(reward_a2c, columns=['reward'])
    results.to_csv("csv_results/{}".format(name_file))

[0]

Episode : 0 rsum=60.95773838536643, 988 actions
Episode : 1 rsum=-38.158907492845834, 999 actions
Episode : 2 rsum=-40.30154751407725, 999 actions
Episode : 3 rsum=-38.09112868813788, 999 actions
Episode : 4 rsum=-39.099594640133446, 999 actions
Episode : 5 rsum=-37.957952573980585, 999 actions
Episode : 6 rsum=-37.97466961714021, 999 actions
Episode : 7 rsum=76.94781983950426, 576 actions
Episode : 8 rsum=-38.24077971982722, 999 actions
Episode : 9 rsum=-38.54809500348983, 999 actions
Episode : 10 rsum=-40.33069700117126, 999 actions
Episode : 11 rsum=-39.29083654870013, 999 actions
Episode : 12 rsum=-38.493530839202634, 999 actions
Episode : 13 rsum=67.43671462716878, 866 actions
Episode : 14 rsum=-39.96773564101788, 999 actions
Episode : 15 rsum=79.51927564579147, 513 actions
Episode : 16 rsum=83.32051975666627, 431 actions
Episode : 17 rsum=72.7669478680508, 711 actions
Episode : 18 rsum=-39.16763063355085, 999 actions
Episode : 19 rsum=-39.11428052665596, 999 actions
Episode : 20 

True

True

True

...

True

True

True

True

Episode : 900 rsum=-15.704214316346206, 999 actions
Episode : 901 rsum=-14.20831012701117, 999 actions
Episode : 902 rsum=89.49107467757317, 631 actions
Episode : 903 rsum=-13.52538518381516, 999 actions
Episode : 904 rsum=-14.899162047552416, 999 actions
Episode : 905 rsum=88.03027529725921, 584 actions
Episode : 906 rsum=-15.027677696924217, 999 actions
Episode : 907 rsum=-14.991454226822524, 999 actions
Episode : 908 rsum=86.57072105416405, 924 actions
Episode : 909 rsum=88.1632318824351, 768 actions
Episode : 910 rsum=-17.093494564985004, 999 actions
Episode : 911 rsum=-13.363711513039165, 999 actions
Episode : 912 rsum=91.02146780541142, 669 actions
Episode : 913 rsum=-15.164842852027492, 999 actions
Episode : 914 rsum=86.53083403343068, 893 actions
Episode : 915 rsum=85.49845771438646, 954 actions
Episode : 916 rsum=-13.507951192592214, 999 actions
Episode : 917 rsum=-13.568685527129988, 999 actions
Episode : 918 rsum=-15.168371366438906, 999 actions
Episode : 919 rsum=87.2773971